In [ ]:
import copy
import os
import sys
import time
from contextlib import contextmanager

import pandas as pd
import numpy as np
import wandb

import nyaggle
from nyaggle.experiment import run_experiment
import lightgbm as lgb


sys.path.append('../')

from src.feature import *
from src.store import *
from src.parser import *
from src.nn import *

@contextmanager
def timer(name):
    s = time.time()
    yield
    elapsed = time.time() - s
    print(f"[{name}] {elapsed:.3f}s")
    
TARGET_COLS = ['target1', 'target2', 'target3', 'target4']
DATA_DIR = '../input/mlb-player-digital-engagement-forecasting'
EXP_NAME = ''
FEATURES = get_feature_list()
TRAIN_FULL = False
USE_UPDATED = True

In [ ]:
with timer('load'):
    store = Store.train(DATA_DIR, use_updated=USE_UPDATED)

In [ ]:
FEATURES = [
    #'f000',
    'f022',
    'f001',
    'f002',
    'f024',
    #'f003',
    #'f004',
    'f023',
    'f005', 
    #'f006',
    'f021',
    #'f007',
    'f014',
    'f037',
    
    'f100',
    'f102',
    'f103',
    'f105',
    'f110',
    'f111',
    'f120',
    'f121',
    'f131',

     'f300',
     'f400',
     'f401',
     'f402',
     'f403',
     'f404',
    'f408',
    'f410',
    
    'f020',
    'f303'
]


In [ ]:
if USE_UPDATED:
    df_train = make_df_base_from_train_engagement(pd.read_feather('../input/mlb-player-digital-engagement-forecasting/train_nextDayPlayerEngagement_updated.f'))
else:
    df_train = make_df_base_from_train_engagement(pd.read_feather('../input/mlb-player-digital-engagement-forecasting/train_nextDayPlayerEngagement.f'))

season_df = pd.read_csv(os.path.join(DATA_DIR, 'seasons.csv'))

In [ ]:
import gc

for lag in [0, 3, 7, 14, 21, 28, 35, 45]:
    X = make_feature(df_train, store, FEATURES, save_to_store=False, with_target=True, lag_requirements=lag, second_order_feature=True)
    X.to_feather(f'X_{lag}_2nd.f')
    del X
    gc.collect()
